In [4]:
import pandas as pd
import numpy as np
import datetime

# used to access folder structures
import os

# used to open images
import PIL

# Graphs, visualizations
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import scipy



from sklearn.model_selection import train_test_split

In [5]:
base_path = "IDC_regular_ps50_idx5/"
folder = os.listdir(base_path)
print("No. of Patients total:",len(folder))

total_images = 0
for n in range(len(folder)):
    patient_id = folder[n]
    for c in [0, 1]:
        patient_path = base_path + patient_id
        class_path = patient_path + '/' + str(c) + '/'
        subfiles = os.listdir(class_path)
        total_images += len(subfiles)
        
print("Total Images in dataset: ", total_images )

No. of Patients total: 279
Total Images in dataset:  277524


In [6]:
# create an empty dataframe with a column for each the patient id,
# the path to the image and the target label for each patch
data = pd.DataFrame(index=np.arange(0, total_images), columns=["patient_id", "path", "target"])

patientData = pd.DataFrame(index=np.arange(0, len(folder)), columns=["patient_id", "nrPos", "nrNeg"])

k = 0
n = 0
# Iterate over all patients (1 folder = 1 patient)
for i in range(len(folder)):
    
    # Fill the patient Data dataframe with the patient and the number of pos and neg patches
    if n > 0:
        patientData.iloc[n-1]["patient_id"] = patient_id
        patientData.iloc[n-1]["nrPos"] = nrPos
        patientData.iloc[n-1]["nrNeg"] = nrNeg
    
    nrPos = 0
    nrNeg = 0
    
    patient_id = folder[n]
    patient_path = base_path + patient_id 
    
    # Iterate over the two subfolders with the negative and positive patches 
    for c in [0,1]:        
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = os.listdir(class_path)
        
        # Iterate over the images in the subfolder and fill the dataframe
        for m in range(len(subfiles)):
            
            # Count the number of positive and negative patches per patient
            if c == 0:
                nrNeg += 1
            else:
                nrPos += 1
            
            image_path = subfiles[m]
            data.iloc[k]["path"] = class_path + image_path
            data.iloc[k]["target"] = c
            data.iloc[k]["patient_id"] = patient_id
            k += 1
    n += 1

# write the last patient
patientData.iloc[n-1]["patient_id"] = patient_id
patientData.iloc[n-1]["nrPos"] = nrPos
patientData.iloc[n-1]["nrNeg"] = nrNeg

In [7]:
data.to_pickle('data.pkl')
patientData.to_pickle('patientData.pkl')